Purpose: Turn text-data into data with appropriate values neccessary for functional RAGAS evaluation

Fetch documents (from only the SciComp wiki as of now) and put into accessable format for generation of expected outputs and creation of context later on

In [ ]:
"""WEB LOADER"""

import argparse
import os
import sys
import logging
import warnings

import bs4 as bs
import html2text
from llama_index.core import Document
from langchain_community.document_loaders import TextLoader

data_path = '../data/janelia.org'  # Use './' to indicate the current directory
text_maker = html2text.HTML2Text()
text_maker.ignore_links = True
text_maker.images_to_alt = True
text_maker.single_line_break = True
text_maker.ignore_emphasis = True
SOURCE = "Web"


def webpage_to_text(soup):
    """ Convert a generic web page to searchable text
    """
    title = soup.title.text
    text = text_maker.handle(str(soup))
    return title,text


def janelia_org_to_text(soup):
    """ Convert a janelia.org page to searchable text
    """
    title = soup.title.text.replace(" | Janelia Research Campus","")
    content_sections = soup.find_all("section", class_="content-section")
    if not content_sections:
        return title,None
    if len(content_sections) > 1:
        raise Exception("More than one content section")
    content = content_sections[0]
    # Remove useless content
    for div in content.find_all("div", {'class':['panels-ipe-label','secondary_menu']}):
        div.decompose()
    # Html2text smashes text together if only tags separate it
    # This fix not only adds the spacing but also adds a separator for nav buttons
    for span in content.find_all("span", {'class':'button-wrapper'}):
        sep = bs.NavigableString(" / ")
        span.insert(0, sep)
    text = text_maker.handle(str(content))
    return title,text


def html_to_text(link, body):
    """ Convert a web page to plain text for use as a GPT prompt.
    """
    soup = bs.BeautifulSoup(body,'lxml')
    if "janelia.org" in link:
        title,text = janelia_org_to_text(soup)
    else:
        title,text = webpage_to_text(soup)
    return title,text


class WebSiteLoader():

    def __init__(self, data_path):
        self.data_path = data_path

    def create_document(self, name, title, link, doc_text):
        metadata = {"source": self.data_path, "title": title, "link": link}
        # Debugging: Print doc_text to ensure it's not empty
        return [Document(page_content=doc_text, metadata=metadata)]
    
    def load_all_documents(self):
        documents = []
        for root, dirs, files in os.walk(self.data_path):
            for name in files:
                filepath = os.path.join(root, name)
                with open(filepath) as f:
                    link = f.readline().strip()
                    body = f.read()
                    title, text = html_to_text(link, body)
                    
                    
                    # print(f"Title: {title}")
                    # print(f"Text: {text}")
                    if text:
                        final_text = title + "\n" + text
                        with open('tempTestGen.txt', 'w') as file:
                            file.write(final_text)
                        loader = TextLoader("./tempTestGen.txt")
                        doc = loader.load()
                        documents.append(doc)
        return documents
    




# Open output.txt in write mode




In [13]:
"""ARCHIVED WIKI LOADRER"""
import argparse
import os
import re
import sys
import logging
import warnings
from langchain_community.document_loaders import TextLoader

import html2text
from llama_index.core import Document

warnings.simplefilter("ignore", ResourceWarning)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# Constants
SOURCE = "Wiki"

text_maker = html2text.HTML2Text()
text_maker.ignore_links = True
text_maker.ignore_images = True


def wiki_to_text(ancestors, title, authors, labels, body):
    """ Convert a wiki document to plain text for use as a GPT prompt.
    """
    body_text = text_maker.handle(body)
    text =  f"Title: {title}\n"
    if authors: text += f"Authors: {authors}\n" 
    if ancestors: text += f"Ancestors: {ancestors}\n" 
    if labels: text += f"Labels: {ancestors}\n"
    text += f"{body_text}"
    return text


class WikiLoader():

    def __init__(self, data_path):
        self.data_path = data_path

    def create_document(self, name, title, link, doc_text):
        metadata = {"source": self.data_path, "title": title, "link": link}
        return [Document(page_content=doc_text, metadata=metadata)]

    def load_all_documents(self):
        documents = []
        for root, dirs, files in os.walk(self.data_path):
            for name in files:
                filepath = os.path.join(root, name)
                with open(filepath) as f:
                    link = f.readline().rstrip()
                    ancestors = f.readline().rstrip()
                    title = f.readline().rstrip()
                    authors = f.readline().rstrip()
                    labels = f.readline().rstrip()
                    body = re.sub('[\n]+', '\n', "".join(f.readlines()))
                    text = wiki_to_text(ancestors, title, authors, labels, body)
                    # doc = self.create_document(name, title, link, text)
                    # documents.append(doc)
                    if text:
                        final_text = title + "\n" + text
                        with open('tempTestGen.txt', 'w') as file:
                            file.write(final_text)
                        loader = TextLoader("./tempTestGen.txt")
                        doc = loader.load()
                        documents.append(doc)
        return documents
    


In [10]:
"""ARCHIVED SLACK LOADER"""
import argparse
import re
import sys
import glob
import json
import logging
import warnings
from decimal import Decimal

from llama_index.core import Document

warnings.simplefilter("ignore", ResourceWarning)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# Constants
SOURCE = "Slack"
DOCUMENT_PAUSE_SECS = 300
IGNORED_SUBTYPES = set(['channel_join','channel_leave','bot_message'])


def get(dictionary, key):
    """ Get the key out of the dictionary, if it exists. If not, return None.
    """
    if dictionary and key in dictionary:
        return dictionary[key]
    return None


def fix_text(text):
    """ Standard transformations on text like squashing multiple newlines.
    """
    text = re.sub("\n+", "\n", text)
    return text


class ArchivedSlackLoader():

    def __init__(self, data_path, debug=False):
        self.data_path = data_path
        self.id2username = {}
        self.id2realname = {}
        self.channel2id = {}
        self.debug = debug

        for user in self.get_users():
            id = user['id']
            self.id2username[id] = user['name']
            self.id2realname[id] = user['profile']['real_name']

        logger.info(f"Loaded {len(self.id2username)} users")
        for channel in self.get_channels():
            logger.debug(f"{channel['id']}: {channel['name']}")
            self.channel2id[channel['name']] = channel['id']
        
        logger.info(f"Loaded {len(self.channel2id)} channels")


    def get_users(self):
        """ Generator which returns users from the users.json file.
        """
        with open(f"{self.data_path}/users.json", 'r') as f:
            users = json.load(f)
            for user in users:
                yield user


    def get_channels(self):
        """ Generator which returns channels from the channels.json file.
        """
        with open(f"{self.data_path}/channels.json", 'r') as f:
            channels = json.load(f)
            for channel in channels:
                yield channel


    def get_messages(self, channel_name):
        """ Generator which returns messages from the json files in the given channel directory.
        """
        for messages_file in glob.glob(f"{self.data_path}/{channel_name}/*.json"):
            with open(messages_file, 'r') as f:
                for message in json.load(f):
                    yield message


    def extract_text(self, elements):
        """ Recursively parse an 'elements' structure, 
            converting user elements to their real names.
        """
        text = ''
        for element in elements:
            if 'elements' in element:
                text += self.extract_text(element['elements'])
            el_type = get(element, 'type')
            if el_type == 'text':
                if get(get(element, 'style'), 'code'): text += '`'
                text += element['text']
                if get(get(element, 'style'), 'code'): text += '`'
            elif el_type == 'link':
                text += get(element, 'url')
            elif el_type == 'rich_text_preformatted':
                text += "\n"
            elif el_type == 'user':
                user_id = element['user_id']
                try:
                    text += self.id2realname[user_id]
                except KeyError:
                    logger.error(f"No such user '{user_id}'")
                    text += user_id

        return text

    def parse_message(self, message):
        """ Parse a message into text that will be read by a GPT model. 
        """
        thread_id, text_msg = None, None
        if get(message, 'type') == 'message':
            if 'subtype' in message and get(message, 'subtype') in IGNORED_SUBTYPES:
                pass
            else:
                ts = message['ts']
                thread_ts = get(message, 'thread_ts') or ts
                thread_id = Decimal(thread_ts)

                # Translate user
                user_id = message['user']
                try:
                    realname = self.id2realname[user_id]
                except KeyError:
                    try:
                        realname = message['user_profile']['display_name']
                    except KeyError:
                        realname = user_id
                    
                if 'blocks' in message:
                    text = self.extract_text(message['blocks'])
                else:
                    text = message['text']
                
                text_msg = re.sub("<@(.*?)>", lambda m: self.id2realname[m.group(1)], text)
                text_msg = fix_text(text_msg)

                if 'attachments' in message:
                    for attachment in message['attachments']:
                        if 'title' in attachment: text_msg += f"\n{fix_text(attachment['title'])}"
                        if 'text' in attachment: text_msg += f"\n{fix_text(attachment['text'])}"
                        
                if 'files' in message:
                    for file in message['files']:
                        if 'name' in file:
                            # There are several cases where a file doesn't have a name:
                            # 1) The file has been deleted (mode=tombstone)
                            # 2) We have no access (file_access=access_denied)
                            text_msg += f"\n<{file['name']}>"

                if 'reactions' in message:
                    text_msg += f"\nOthers reacted to the previous message with "
                    r = [f"{reaction['name']} a total of {reaction['count']} times" for reaction in message['reactions']]
                    text_msg += ", and with ".join(r) + "."

                text_msg = f"{realname} said: {text_msg}\n"
        
        return thread_id, text_msg


    def create_document(self, channel_id, ts, doc_text):
        logger.info("--------------------------------------------------")
        logger.info(f"Document[channel={channel_id},ts={ts}]")
        logger.debug(doc_text)
        return Document(text=doc_text, extra_info={"source": SOURCE, "channel": channel_id, "ts": ts})


    def load_documents(self, channel_name):
        channel_id = self.channel2id[channel_name]
        messages = {}
        for message in self.get_messages(channel_name):
            try:
                thread_id, text_msg = self.parse_message(message)
            except Exception as e:
                logger.error(f"Error parsing message: {message}")
                raise e
                
            if thread_id and text_msg:
                if thread_id not in messages:
                    messages[thread_id] = []
                messages[thread_id].append(text_msg)

        prev_id = Decimal(0)
        documents = []
        doc_text = ""
        start_ts = None

        for thread_id in sorted(list(messages.keys())):

            # Create a new document whenever messages are separated by a longer pause
            if doc_text and thread_id-prev_id > DOCUMENT_PAUSE_SECS:
                doc = self.create_document(channel_id, start_ts, doc_text)
                documents.append(doc)
                doc_text = ""
                start_ts = None

            logger.debug(thread_id)

            # Starting timestamp for the next document
            if not start_ts:
                start_ts = str(thread_id)

            # Add all messages from the current thread
            for text_msg in messages[thread_id]:
                doc_text += text_msg

            prev_id = thread_id

        # Add final document
        doc = self.create_document(channel_id, start_ts, doc_text)
        documents.append(doc)

        return documents


    def load_all_documents(self):
        documents = []
        for channel_name in self.channel2id.keys():
            for doc in self.load_documents(channel_name):
                documents.append(doc)
        return documents

data_path = '../data/slack/janelia-software/slack_to_2023-05-18'
loader = ArchivedSlackLoader(data_path)
documents = loader.load_all_documents()
with open('documents.txt', 'w') as file:
    for document in documents:
        file.write(str(document) + '\n')


In [11]:
"""DOCUMENT LOADER ALL SOURCES"""
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# generator with openai models
from dotenv import load_dotenv
from langchain_community.llms import Ollama
import pandas as pd

import os
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
from langchain_community.document_loaders import TextLoader
import os

# DONE: Recursively load all scraped files in the directory and its subdirectories
# loader = TextLoader("./test.txt")
# documents = loader.load()
# print(documents)

from langchain_community.document_loaders import TextLoader
import os
import glob

import glob
import os
import json
import mimetypes
from bs4 import BeautifulSoup

class DocumentLoader:
    def __init__(self, root_dir):
        self.root_dir = root_dir

    def load_documents(self):
        all_documents = []
        for folder_name in os.listdir(self.root_dir):
            folder_path = os.path.join(self.root_dir, folder_name)
            if os.path.isdir(folder_path):
                if folder_name == "wiki":
                    loader = WikiLoader(folder_path)
                # elif folder_name == "slack":
                #     loader = ArchivedSlackLoader(folder_path)
                elif folder_name == "janelia.com":
                    loader = WebSiteLoader(folder_path)
                else:
                    continue  # Skip if folder doesn't match any criteria
                documents = loader.load_all_documents()
                all_documents.extend(documents)
        return all_documents

# Assuming your data folder is at "./data/"
loader = DocumentLoader("../data")
documents = loader.load_documents()
print (documents)
# Assuming your data folder is at "../data/"
with open('documents.txt', 'w') as file:
    for document in documents:
        file.write(str(document) + '\n')
# Assuming documents is a list of strings or convertible to string


# Now `final_df` contains all the generated testsets in one DataFrame

INFO:datasets:PyTorch version 2.3.1 available.


/Users/bakhalea/Documents/gpt-semantic-search/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[]


In [ ]:
generator_llm = Ollama(model="llama3")
critic_llm = Ollama(model="llama3")

# generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo")

embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
) 



# Initialize an empty list to collect all testsets
all_testsets = []

# Iterate over each document
for document in documents:
    # Generate a testset for the current document
    current_testset = generator.generate_with_langchain_docs(document, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})
    
    # Assuming `current_testset` is a list of dictionaries where each dictionary represents a row
    # Convert the current testset to a DataFrame and append it to the list
    all_testsets.append(pd.DataFrame(current_testset))

# Concatenate all DataFrames in the list into a single DataFrame
final_df = pd.concat(all_testsets, ignore_index=True)
df = pd.read_parquet('testset.parquet')

Generate the testset

In [ ]:


# import logging
# import sys
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger('llama_index').setLevel(logging.DEBUG)
# logging.getLogger('openai').setLevel(logging.DEBUG)

# logger = logging.getLogger(__name__)
# logger.setLevel(logging.DEBUG)


#change the model to a better one once the whole thing is functional



In [ ]:

# df = testset.to_pandas()


Address potentiall duplicate questions (which the testset generator has done) and delete their rows

In [ ]:

df = df.drop_duplicates(subset='question', keep='first')
questions_list = df['question'].tolist()
print (questions_list)

seen = set()
questions_list = [x for x in questions_list if not (x in seen or seen.add(x))]

for item in questions_list:
    print(item)

    

# testset.to_json("testset.json")
# Creates dataset of ground truths contexts and questions for the testset
# Missing answers column 
# On one medium size document, it took about 1 minutes to generate 10 questions and cost 3 dollars on OpenAI
# Seems expensive when using gpt-4, is its use justified or does 3.5 get the job done?
# Evaluate gpt-4 vs gpt-3.5-turbo-16k for RAGAS evaluation test data generation


fetch the LLM's response and append to a list

In [ ]:
# Assuming SemanticSearchParser is your class and it has a method named 'generate_response' that takes a question and returns an answer

# Initialize your SemanticSearchParser class
# Adjust this step if your class initialization requires different parameters


# Now you can import the class

from generate_answer import SemanticSearchService

weaviate_url = "http://localhost:8777"
service = SemanticSearchService(weaviate_url)
print (questions_list)

# List to store answers (optional)
answers_list = []


# Loop through each question in the questions_list
for question in questions_list:
    # Use the question as input to get the answer
    answer = service.generate_response(question)
    
    # Print the answer
    
    # Optionally, append the answer to answers_list for further processing
    answers_list.append(answer)

print (answers_list)
# temp_ans_list = ["The Janelia Scientific Computing team operates and maintains a world-class computational infrastructure that includes a high-performance compute cluster with over 5000 cores and 300 GPUs. This infrastructure is used to analyze and mine the large amounts of data produced by Janelia's scientists. The team also supports a state-of-the-art storage and compute infrastructure across two data centers, which currently supports over 15 petabytes of scientific data. This data is split across various storage tiers and is connected with an optical fiber ring. The team also maintains a 4500 sq ft data center with significant power and cooling capacity. \n\nIn addition to hardware, the team also has deep software skills in a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. These skills are used to help with research and engineering tasks, from quick questions to full software life cycle support. The team's software skills, combined with their domain knowledge in areas such as image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics, allow them to efficiently work with both experimentalists and computer scientists. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are used for various aspects of data analysis and simulation in biological research.", "The Janelia Scientific Computing team provides a wide range of support for advanced imaging techniques and image analysis. They offer consultation on experiment design as well as image visualization and processing. They also provide comprehensive image and data analysis support for multiple software packages through hands-on assistance and/or custom-written macros/plugins/scripts for ImageJ/FIJI, MATLAB, Imaris, etc. \n\nIn addition, they maintain several computer workstations dedicated to viewing and processing large image datasets acquired with the facility's instruments. These workstations are equipped with a suite of imaging software, including a full version of Imaris, and have robust hardware specifications to handle large datasets. \n\nThe team also has deep domain knowledge in image processing, machine learning, data handling, and 3D graphics & visualization, which allows them to efficiently work with experimentalists and computer scientists in various research areas.", "The Janelia Scientific Computing team provides world-class computational infrastructure to support the institute's scientific endeavors. They operate and maintain all of Janelia’s storage and associated backup infrastructure, high performance compute cluster, and all Linux systems. They also manage Janelia’s data center and backup and disaster recovery resources. The team supports a Linux compute cluster with over 5000 cores and 300 GPUs, and is responsible for maintaining many other Linux servers and workstations. They also handle a significant amount of data, with almost 100TB of new Janelia’s data being safely backed up every month.\n\nIn addition to infrastructure, the Scientific Computing Software team works closely with Janelia's labs and project teams, providing everything from answering quick questions to full software life cycle support. They have a broad range of software skills, including programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in areas like image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics. \n\nThe team also identifies opportunities for code reuse, reducing development overhead and support costs across Janelia. They are strong proponents of open science and have created the Open Science Software Initiative. Most of their software is open source and available via GitHub. They also run a Scientific Computing Associates program to embed associates in SciComp and the lab or team they work with. \n\nThe team is led by Stephan Preibisch and consists of three teams: Software Engineering headed by Konrad Rokicki, Computational Methods and Solutions, both headed by Stephan Preibisch. They have developed several tools and projects like NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. \n\nIn summary, the Janelia Scientific Computing team supports the institute's mission and drives innovation in modern biological research by providing robust computational infrastructure, software support, and developing innovative tools and solutions.", 'The Janelia Scientific Computing team supports advanced imaging techniques in neuroscience and cell biology through a variety of ways. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. They also develop and maintain a range of software tools and applications that aid in these areas. Some of these tools include NeuronBridge for finding neuron matches across modalities, HortaCloud for cloud-based collaborative annotation, VVD Viewer for volumetric rendering of 3D/4D microscopy data, and BigStitcher for efficient alignment of multi-tile and multi-angle image datasets. They also work closely with labs and project teams, providing full software life cycle support.', "The Janelia Scientific Computing team supports modern biological research in several ways. They maintain a world-class computational infrastructure, including storage and backup infrastructure, a high-performance compute cluster, and all Linux systems. They also manage Janelia's data center and backup and disaster recovery resources. The team supports data storage infrastructure for storing and accessing scientific data, with over 15 petabytes of scientific data split across various storage tiers. They also support a Linux compute cluster with over 5000 cores and 300 GPUs, and maintain many other Linux servers and workstations. \n\nIn addition to infrastructure support, the Scientific Computing Software team works closely with Janelia's labs, project teams, and shared resources to help with research and engineering tasks. They provide everything from answering quick questions to full software life cycle support. The team's software skills span a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. They are strong proponents of open science and most of their software is open source and available via GitHub. They also run the Scientific Computing Associates program, which offers challenging assignments for those interested in computational science.", "The Janelia Scientific Computing team provides comprehensive support for advanced imaging techniques in neuroscience, cell biology, and bioinformatics through a variety of collaborations and custom software tools. They work closely with Janelia's labs, project teams, and shared resources to assist with research and engineering tasks. This can range from answering quick questions to providing full software life cycle support.\n\nThe team's software skills cover a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. Many team members have backgrounds in biology, enabling them to work efficiently with experimentalists and computer scientists.\n\nThe team is also involved in various projects and tools such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are designed to support advanced imaging techniques and data analysis in neuroscience, cell biology, and bioinformatics.\n\nFurthermore, the team is a strong proponent of open science and has teamed up with the Computation & Theory research area to create the Open Science Software Initiative. Most of their software is open source and available via GitHub, promoting collaboration and knowledge sharing. They also run the Scientific Computing Associates program, which embeds associates in SciComp and the lab or team they work with."]




Add values from the list of JaneliaGPT responses to the dataframe["answer"]

In [ ]:
df['answer'] = None
# Assuming df is your DataFrame and answers_list is a list with values to populate the 'Answer' column
if len(df) == len(answers_list):
    df['answer'] = answers_list
else:
    print("The length of answers_list does not match the number of rows in the DataFrame.")

df.dropna(axis=1, how='all', inplace=True)
display(df)
# df.to_json('WithAnswersDatasetFromTestTxt.json', index=True)


Preprocess the dataframe to conver to a Dataset

In [ ]:
#Attempt to fix the issue with the answers_list not bsjdkflasdjjfklasdkl fasdjkfh jkas
df_fix = df
# List of columns to keep
columns_to_keep = ['question', 'ground_truth', 'answer', 'contexts']

# Reassign df to a DataFrame containing only the columns to keep

df_fix= df_fix[columns_to_keep]



# Assuming df is already defined and contains the necessary columns

# Convert 'question' and 'answer' to lists of strings if they are not already
# df_fix['question'] = df_fix['question'].apply(lambda x: [x] if isinstance(x, str) else x)
# df_fix['answer'] = df_fix['answer'].apply(lambda x: [x] if isinstance(x, str) else x)

# Ensure 'contexts' and 'ground_truth' are lists of lists of strings
# This step assumes 'contexts' and 'ground_truth' are already in the correct format
# If not, you would need to apply a similar conversion as above, ensuring each element is a list

# Example conversion if 'contexts' and 'ground_truth' were not already lists of lists
df_fix['contexts'] = df_fix['contexts'].apply(lambda x: [[y] for y in x] if isinstance(x, list) and all(isinstance(y, str) for y in x) else x)
df_fix['ground_truth'] = df_fix['ground_truth'].apply(lambda x: [[y] for y in x] if isinstance(x, list) and all(isinstance(y, str) for y in x) else x)
display(df_fix)
# Now df should be in the correct format for training

from datasets import load_dataset

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")

from datasets import Dataset
dataset_fix = Dataset.from_pandas(df_fix)
dataset_fix = dataset_fix.remove_columns(['__index_level_0__'])


Convert dataframe to Dataset and compare to a vaild Dataset for analysis

In [ ]:
from datasets import Features

# Assuming dataset_fix and amnesty_qa["eval"] are your datasets
features_dataset_fix = dataset_fix.features
# features_amnesty_eval = amnesty_qa["eval"].features
def format_columns(example):
    # Format 'question', 'answer', and 'ground_truths' columns to single values
    for column in ['ground_truth', 'answer', 'question']:
        if column in example and example[column]:
            example[column] = example[column]
    
    # Correctly format 'contexts' column to a list of list of strings
    if 'contexts' in example:
        # Ensure 'contexts' is a list of strings (not a list of lists)
        if isinstance(example['contexts'], list):
            # If the items are lists (or other non-string types), flatten and convert to strings
            example['contexts'] = [str(item) for sublist in example['contexts'] for item in (sublist if isinstance(sublist, list) else [sublist])]
        else:
            # If 'contexts' is not a list, convert it into a list of a single string
            example['contexts'] = [str(example['contexts'])]

    
    return example


# Apply the transformation to both datasets
dataset_fix = dataset_fix.map(format_columns)
# Direct comparison of data types
"""if set(features_dataset_fix.keys()) == set(features_amnesty_eval.keys()):
    all_types_match = True
    for key in features_dataset_fix.keys():
        type_dataset_fix = type(features_dataset_fix[key]).__name__
        type_amnesty_eval = type(features_amnesty_eval[key]).__name__
        if type_dataset_fix != type_amnesty_eval:
            print(f"Data type for feature '{key}' differs between datasets. dataset_fix: {type_dataset_fix}, amnesty_qa['eval']: {type_amnesty_eval}")
            all_types_match = False
    if all_types_match:
        print("The data types of all features in both datasets match.")
else:
    print("The Features of the datasets differ in their keys.")"""
"""
print (dataset_fix["question"])
print (dataset_fix["answer"])
print (dataset_fix["ground_truth"])
print (dataset_fix["contexts"])
"""



In [ ]:
"""from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

contextual_precision = ContextualPrecisionMetric()
contextual_recall = ContextualRecallMetric()
contextual_relevancy = ContextualRelevancyMetric()

from deepeval.test_case import LLMTestCase
from deepeval import evaluate
modified_items = []

# Step 2: Iterate over each item in dataset_fix
for item in dataset_fix:
    # Step 3: Create a new LLMTestCase instance with modified fields
    modified_item = LLMTestCase(
        input=item["question"],
        actual_output=item["answer"],
        expected_output=item["ground_truth"],
        retrieval_context=item["contexts"]
    )
    # Step 4: Append the modified item to the list
    modified_items.append(modified_item)
# Assuming dataset_fix supports item assignment



evaluate(
    test_cases=[modified_items],
    metrics=[contextual_precision, contextual_recall, contextual_relevancy]
)

"""

Run evaluation to gather the below metrics

In [ ]:

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)


from ragas import evaluate

eval_llm = Ollama(model="llama3")
# Will reutrn a dataframe with the metrics
# Returns error for now because answers column is missing
# Error is misleading, fix dataset first and make it match the docs example dataset

result = evaluate(
    dataset_fix,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=eval_llm,
)

result



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
result.to_pandas()



In [ ]:
import pandas as pd
import plotly.express as px
import json
import os

class MetricsManager:
    def __init__(self, file_path='metrics.json'):
        self.file_path = file_path
        self.metrics = self.load_metrics()

    def load_metrics(self):
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as file:
                return json.load(file)
        else:
            return {}

    def save_metrics(self):
        with open(self.file_path, 'w') as file:
            json.dump(self.metrics, file, indent=4)

    def update_metrics(self, trial_name, context_precision, faithfulness, answer_relevancy, context_recall):
        averages = {
            'context_precision': context_precision,
            'faithfulness': faithfulness,
            'answer_relevancy': answer_relevancy,
            'context_recall': context_recall
        }
        self.metrics[trial_name] = averages
        self.save_metrics()

    def render_table(self):
        if not self.metrics:
            print("No metrics available for plotting.")
            return

        trials, metrics, averages = [], [], []
        for trial_name, metrics_averages in self.metrics.items():
            for metric, average in metrics_averages.items():
                trials.append(trial_name)
                metrics.append(metric)
                averages.append(round(average, 4))

        Eval_Categories = pd.DataFrame({
            'Trial': trials,
            'Metric': metrics,
            'Average': averages
        })

        if Eval_Categories.empty:
            print("No metrics data to plot.")
            return

        fig = px.bar(
            Eval_Categories,
            x='Trial',
            y='Average',
            color='Metric',
            barmode='group',
            text='Average',
            category_orders={"Metric": ["context_precision", "faithfulness", "answer_relevancy", "context_recall"]},
            labels={
                "Average": "Average Score",
                "Metric": "Metric",
                "Trial": "Trial Name"
            }
        )

        fig.update_layout(
            width=1000,
            height=600,
            title="<b>Average Scores of Evaluation Metrics by Trial</b>",
            xaxis_title="Trial Name",
            yaxis_title="Average Score",
            font=dict(size=15)
        )

        fig.show()

MetricsManager()